In [1]:
import sys
import subprocess
sys.path.append('f:/seismic/code/core')
sys.path.append('/seismic/code/core')
sys.path.append('/kaggle/input/my-seismic-library/')
import kaggle_support as kgs
import importlib
import seis_model
import time
import pandas as pd
import numpy as np
kgs.cache_dir_write = kgs.cache_dir_write+'/../cache_evaluate/'
kgs.cache_dir_read = kgs.cache_dir_write

F:/seismic/models/brendan/*.pth
Loading:  F:/seismic/models/brendan\unet2d_caformer_seed3_epochbest.pt
n_models: 1


In [2]:
# Set up model
importlib.reload(seis_model)
model = seis_model.default_model()
model.write_cache = False

In [ ]:
data = kgs.load_all_train_data(validation_only=True)[::25]
model.models[0].batch_size = 1
families = list(set([d.family for d in data]))
families.sort()
res = dict()
res['family'], res['score_brendan'], res['score_mine'], res['score_mine_max'], res['time_taken'] = [],[],[],[],[]
for f in families:
    res['family'].append(f)
    this_data = []
    for d in data:
        if d.family==f:
            this_data.append(d)
    data_out_brendan = model.models[0].infer(this_data)
    res['score_brendan'].append(kgs.score_metric(data_out_brendan,show_diagnostics=False)[0])
    data_out = model.infer(this_data[:1])
    t=time.time()
    data_out = data_out+model.infer(this_data[1:])
    res['time_taken'].append((time.time()-t)/len(this_data[1:]))
    score,_,res_all = kgs.score_metric(data_out,show_diagnostics=False)
    res['score_mine'].append(score)
    res['score_mine_max'].append(np.max(res_all))
    print(pd.DataFrame(res))
df = pd.DataFrame(res)
print('Brendan local score: ', np.mean(df['score_brendan']))
print('My local score: ', np.mean(df['score_mine']))
print('Estimate public LB: ', 28.8*np.mean(df['score_mine'])/np.mean(df['score_brendan']))
df

Inferring: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 391.69it/s]


         family  score_brendan  score_mine  score_mine_max  time_taken
0  CurveFault_A       3.510337    3.510337        6.826997    0.007419


Inferring: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 391.68it/s]


         family  score_brendan  score_mine  score_mine_max  time_taken
0  CurveFault_A       3.510337    3.510337        6.826997    0.007419
1  CurveFault_B      78.843337   78.843337      151.991837    0.007451


Inferring: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 383.78it/s]


...

Inferring: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 351.50it/s]


         family  score_brendan  score_mine  score_mine_max  time_taken
0  CurveFault_A       3.510337    3.510337        6.826997    0.007419
1  CurveFault_B      78.843337   78.843337      151.991837    0.007451
2    CurveVel_A       8.350388    8.350388       15.488571    0.007738
3    CurveVel_B      41.801908   41.801908      174.948571    0.009317
4   FlatFault_A       2.969474    2.969474       20.227192    0.007760
5   FlatFault_B      28.441020   28.441020      105.257551    0.008268


Inferring:   0%|                                                                                | 0/18 [00:00<?, ?it/s]

capturing graph
capturing graph


Inferring:   6%|████                                                                    | 1/18 [00:18<05:17, 18.66s/it]